# TensorFlow Tutorial

## MNIST for experts
<https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html>  
CNNを用いたネットワークでMNISTをクラス分類. 99%程度の認識率.

### 初期準備
本コードを実行する前にtensorflow tutorialに付随するinput_data.pyを同じディクショナリに用意する
<https://tensorflow.googlesource.com/tensorflow/+/master/tensorflow/examples/tutorials/mnist/input_data.py
>

In [ ]:
# TensowFlowのインポート
import tensorflow as tf
# mnistデータの用意
import input_data

import time

# 開始時刻
start_time = time.time()
print ("開始時刻: " + str(start_time))

### 関数の定義

In [ ]:
def weight_variable(shape):
    """適度にノイズを含んだ（対称性の除去と勾配ゼロ防止のため）重み行列作成関数
    """

    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """バイアス行列作成関数
    """
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    """2次元畳み込み関数
       strides 真ん中2つが縦横のストライド
    """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """2x2マックスプーリング関数
       strides 真ん中2つが縦横のストライド
    """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### MNISTデータセットの準備
- 60000点の訓練データ（mnist.train）と10000点のテストデータ（mnist.test）がある
- 訓練データとテストデータにはそれぞれ0-9の画像とそれに対応するラベル（0-9）がある
- 画像は28x28px(=784)のサイズ
- mnist.train.imagesは[60000, 784]の配列であり、mnist.train.lablesは[60000, 10]の配列
- lablesの配列は、対応するimagesの画像が3の数字であるならば、[0,0,0,1,0,0,0,0,0,0]となっている
- mnist.test.imagesは[10000, 784]の配列であり、mnist.test.lablesは[10000, 10]の配列
![](https://www.tensorflow.org/versions/r0.10/images/MNIST-Matrix.png)
![](https://www.tensorflow.org/versions/r0.10/images/mnist-train-xs.png)
![](https://www.tensorflow.org/versions/r0.10/images/mnist-train-ys.png)

In [ ]:
print("--- MNISTデータの読み込み開始 ---")
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print("mnist.train: ", end="")
print(mnist.train.images.shape)
print(mnist.train.images)
print("mnist.test: ", end="")
print(mnist.train.images.shape)
print(mnist.test.images)
print("--- MNISTデータの読み込み完了 ---")

### Sessionの定義

In [ ]:
sess = tf.InteractiveSession()

### TensorFlowの変数を定義

`with tf.device("/cpu:0"):`と記述し後の変数やモデルを定義するコードをこのwith内に含めると, コードを走らせるデバイスを指定可能.

In [ ]:
# データ用可変2階テンソルを用意
x = tf.placeholder("float", shape=[None, 784])
# 正解用可変2階テンソルを用意
y_ = tf.placeholder("float", shape=[None, 10])

# 画像をリシェイプ 第2引数は画像数(-1は元サイズを保存するように自動計算)、縦x横、チャネル
x_image = tf.reshape(x, [-1, 28, 28, 1])

### 学習のモデルを定義
![](https://qiita-image-store.s3.amazonaws.com/0/127038/fdfb6202-17d7-3d8f-bf03-f2fa89cab4a2.png)

In [ ]:
### 1層目 畳み込み層

# 畳み込み層のフィルタ重み、引数はパッチサイズ縦、パッチサイズ横、入力チャネル数、出力チャネル数
# 5x5フィルタで32チャネルを出力（入力は白黒画像なので1チャンネル）
W_conv1 = weight_variable([5, 5, 1, 32])
# 畳み込み層のバイアス
b_conv1 = bias_variable([32])
# 活性化関数ReLUでの畳み込み層を構築
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

### 2層目 プーリング層

# 2x2のマックスプーリング層を構築
h_pool1 = max_pool_2x2(h_conv1)

### 3層目 畳み込み層

# パッチサイズ縦、パッチサイズ横、入力チャネル、出力チャネル
# 5x5フィルタで64チャネルを出力
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

### 4層目 プーリング層
h_pool2 = max_pool_2x2(h_conv2)

### 5層目 全結合層

# オリジナル画像が28x28で、今回畳み込みでpadding='SAME'を指定しているため
# プーリングでのみ画像サイズが変わる。2x2プーリングで2x2でストライドも2x2なので
# 縦横ともに各層で半減する。そのため、28 / 2 / 2 = 7が現在の画像サイズ

# 全結合層にするために、1階テンソルに変形。画像サイズ縦と画像サイズ横とチャネル数の積の次元
# 出力は1024（この辺は決めです）　あとはSoftmax Regressionと同じ
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# ドロップアウトを指定
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 6層目 Softmax Regression層

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


### 誤差の計算

In [ ]:
#   クロスエントロピーの計算をname_scopeでまとめる
with tf.name_scope("cross-entropy") as scope:
        cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

#   最急降下法の計算をname_scopeでまとめる
 with tf.name_scope("training") as scope:
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

### 予測と認識結果の計算

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

### TensorFlowのおまじない

In [ ]:
# 意した変数Veriableの初期化を実行する
sess.run(tf.initialize_all_variables())

### TensorBoeard用の準備

In [ ]:
# TensorBoardにまとめる項目の設定
    tf.scalar_summary("lossの変化", cross_entropy)

    # TensorBoardにこのネットワークのGraphを描画できるように
    # TensorBoardで表示する値の設定
    summary_op = tf.merge_all_summaries()
    summary_writer = tf.train.SummaryWriter('tb_mnist_experts', graph=sess.graph)

### 予測と認識結果の計算

### 学習ステップ毎の処理

In [ ]:
for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

        # 1step毎にTensorBoardに表示する値を追加する
        summary_str = sess.run(summary_op, feed_dict={x :batch[0], y_: batch[1], keep_prob: 0.5})
        summary_writer.add_summary(summary_str, i)

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

### 結果の表示

In [ ]:
print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

    # 終了時刻
    end_time = time.time()
    print("終了時刻: " + str(end_time))
    print("かかった時間: " + str(end_time - start_time))